In [8]:
import scapy
from scapy.all import *
from scapy.utils import PcapReader

def findAllFile(base):
   for root,dirs,files in os.walk(base):
        for file in files:
            #获取文件路径
            path = os.path.join(root,file)
            new_path = path.replace('flow','edit_data')
findAllFile(r'E:\flow')

In [10]:
import os
import dpkt
def get_payload(buf):
    eth = dpkt.ethernet.Ethernet(buf)
    if not isinstance(eth.data, dpkt.ip.IP): #解包，网络层，判断网络层是否存在，
        return None
    ip = eth.data
    if not isinstance(ip.data, (dpkt.tcp.TCP,dpkt.udp.UDP)): #解包，判断传输层协议是否是TCP，即当你只需要TCP时，可用来过滤
        return  None
    transfer_data = ip.data
    payload = transfer_data.data
    if not len(payload):
        return None
    try:
        dns = dpkt.dns.DNS(payload)
        return None
    except:
        return payload.hex()



# for ts,app_data in all_pcap_data_hex.items():
#     #打印最后一个包的十六进制形式，因为加密数据在命令行打印会出现大量乱码和错行，故在此不做演示打印包的字符形式
#     print()
#     print (app_data)


def clean_data(path):
    for root,dirs,files in os.walk(path):
        for file in files:
            #获取文件路径
            f_path = os.path.join(root,file)
            new_path = f_path.replace('flow','edit_data')
            try:
                f = open(f_path,mode='rb')
                w = open(new_path,mode = 'wb')
                pcap = dpkt.pcap.Reader(f)
                writer = dpkt.pcap.Writer(w)
                for (ts,pkt) in pcap:
                        eth = dpkt.ethernet.Ethernet(pkt) #解包，物理层
                        # if not isinstance(eth.data, dpkt.ip.IP): #解包，网络层，判断网络层是否存在，
                        #     continue
                        # ip = eth.data
                        # if not isinstance(ip.data, (dpkt.tcp.TCP,dpkt.udp.UDP)): #解包，判断传输层协议是否是TCP，即当你只需要TCP时，可用来过滤
                        #     continue
                        # # if not isinstance(ip.data, dpkt.udp.UDP):#解包，判断传输层协议是否是UDP
                        # # 	continue
                        # transf_data = ip.data #传输层负载数据，基本上分析流量的人都是分析这部分数据，即应用层负载流量
                        # if not len(transf_data.data): #如果应用层负载长度为0，即该包为单纯的tcp包，没有负载，则丢弃
                        #     continue
                        # all_pcap_data[ts]=transf_data.data #将时间戳与应用层负载按字典形式有序放入字典中，方便后续分析.
                        # all_pcap_data_hex[ts]=transf_data.data.hex()
                        payload = get_payload(pkt)
                        if payload is not None :
                            writer.writepkt(pkt,ts)
            except:
                 print(new_path)
            f.close()
            w.close()

clean_data(r'E:\flow')

E:\edit_data\a\a_1572884111.pcap
E:\edit_data\a\a_1572920736.pcap
E:\edit_data\a\a_1572931966.pcap
E:\edit_data\a\a_1572938981.pcap
E:\edit_data\a\a_1572972219.pcap
E:\edit_data\a\a_1573002091.pcap
E:\edit_data\a\a_1573005611.pcap
E:\edit_data\a\a_1573024814.pcap
E:\edit_data\a\a_1573044045.pcap
E:\edit_data\a\a_1573111644.pcap
E:\edit_data\a\a_1573113647.pcap
E:\edit_data\a\a_1573125137.pcap
E:\edit_data\a\a_1573138658.pcap
E:\edit_data\a\a_1573142606.pcap
E:\edit_data\a\a_1573250721.pcap
E:\edit_data\a\a_1573255622.pcap
E:\edit_data\a\a_1573260603.pcap
E:\edit_data\a\a_1573298059.pcap
E:\edit_data\a\a_1572867813.pcap_Flow.csv
E:\edit_data\f\f_1572973950.pcap
E:\edit_data\f\f_1573020058.pcap
E:\edit_data\f\f_1573029712.pcap
E:\edit_data\f\f_1573042322.pcap
E:\edit_data\f\f_1573092245.pcap
E:\edit_data\f\f_1573130066.pcap
E:\edit_data\f\f_1573131053.pcap
E:\edit_data\f\f_1573213557.pcap
E:\edit_data\f\f_1573222604.pcap
E:\edit_data\f\f_1573266895.pcap
E:\edit_data\raw\a.zip
E:\edit_dat

In [10]:
from pathlib import Path
from scapy.layers.dns import DNS
from scapy.layers.inet import TCP
from scapy.packet import Padding
from scapy.utils import rdpcap
def read_pcap(path: Path):
    packets = rdpcap(str(path))

    return packets


def should_omit_packet(packet):
    # SYN, ACK or FIN flags set to 1 and no payload
    if TCP in packet and (packet.flags & 0x13):
        # not payload or contains only padding
        layers = packet[TCP].payload.layers()
        if not layers or (Padding in layers and len(layers) == 1):
            return True

    # DNS segment
    if DNS in packet:
        return True

    return False
# for app identification
PREFIX_TO_APP_ID = {
    # AIM chat
    'aim_chat_3a': 0,
    'aim_chat_3b': 0,
    'aimchat1': 0,
    'aimchat2': 0,
    # Email
    'email1a': 1,
    'email1b': 1,
    'email2a': 1,
    'email2b': 1,
    # Facebook
    'facebook_audio1a': 2,
    'facebook_audio1b': 2,
    'facebook_audio2a': 2,
    'facebook_audio2b': 2,
    'facebook_audio3': 2,
    'facebook_audio4': 2,
    'facebook_chat_4a': 2,
    'facebook_chat_4b': 2,
    'facebook_video1a': 2,
    'facebook_video1b': 2,
    'facebook_video2a': 2,
    'facebook_video2b': 2,
    'facebookchat1': 2,
    'facebookchat2': 2,
    'facebookchat3': 2,
    # FTPS
    'ftps_down_1a': 3,
    'ftps_down_1b': 3,
    'ftps_up_2a': 3,
    'ftps_up_2b': 3,
    # Gmail
    'gmailchat1': 4,
    'gmailchat2': 4,
    'gmailchat3': 4,
    # Hangouts
    'hangout_chat_4b': 5,
    'hangouts_audio1a': 5,
    'hangouts_audio1b': 5,
    'hangouts_audio2a': 5,
    'hangouts_audio2b': 5,
    'hangouts_audio3': 5,
    'hangouts_audio4': 5,
    'hangouts_chat_4a': 5,
    'hangouts_video1b': 5,
    'hangouts_video2a': 5,
    'hangouts_video2b': 5,
    # ICQ
    'icq_chat_3a': 6,
    'icq_chat_3b': 6,
    'icqchat1': 6,
    'icqchat2': 6,
    # Netflix
    'netflix1': 7,
    'netflix2': 7,
    'netflix3': 7,
    'netflix4': 7,
    # SCP
    'scp1': 8,
    'scpdown1': 8,
    'scpdown2': 8,
    'scpdown3': 8,
    'scpdown4': 8,
    'scpdown5': 8,
    'scpdown6': 8,
    'scpup1': 8,
    'scpup2': 8,
    'scpup3': 8,
    'scpup5': 8,
    'scpup6': 8,
    # SFTP
    'sftp1': 9,
    'sftp_down_3a': 9,
    'sftp_down_3b': 9,
    'sftp_up_2a': 9,
    'sftp_up_2b': 9,
    'sftpdown1': 9,
    'sftpdown2': 9,
    'sftpup1': 9,
    # Skype
    'skype_audio1a': 10,
    'skype_audio1b': 10,
    'skype_audio2a': 10,
    'skype_audio2b': 10,
    'skype_audio3': 10,
    'skype_audio4': 10,
    'skype_chat1a': 10,
    'skype_chat1b': 10,
    'skype_file1': 10,
    'skype_file2': 10,
    'skype_file3': 10,
    'skype_file4': 10,
    'skype_file5': 10,
    'skype_file6': 10,
    'skype_file7': 10,
    'skype_file8': 10,
    'skype_video1a': 10,
    'skype_video1b': 10,
    'skype_video2a': 10,
    'skype_video2b': 10,
    # Spotify
    'spotify1': 11,
    'spotify2': 11,
    'spotify3': 11,
    'spotify4': 11,
    # Torrent
    'torrent01': 12,
    # Tor
    'torfacebook': 13,
    'torgoogle': 13,
    'tortwitter': 13,
    'torvimeo1': 13,
    'torvimeo2': 13,
    'torvimeo3': 13,
    'toryoutube1': 13,
    'toryoutube2': 13,
    'toryoutube3': 13,
    # Vimeo
    'vimeo1': 14,
    'vimeo2': 14,
    'vimeo3': 14,
    'vimeo4': 14,
    # Voipbuster
    'voipbuster1b': 15,
    'voipbuster2b': 15,
    'voipbuster3b': 15,
    'voipbuster_4a': 15,
    'voipbuster_4b': 15,
    # Youtube
    'youtube1': 16,
    'youtube2': 16,
    'youtube3': 16,
    'youtube4': 16,
    'youtube5': 16,
    'youtube6': 16,
    'youtubehtml5_1': 16,
}

ID_TO_APP = {
    0: 'AIM Chat',
    1: 'Email',
    2: 'Facebook',
    3: 'FTPS',
    4: 'Gmail',
    5: 'Hangouts',
    6: 'ICQ',
    7: 'Netflix',
    8: 'SCP',
    9: 'SFTP',
    10: 'Skype',
    11: 'Spotify',
    12: 'Torrent',
    13: 'Tor',
    14: 'Vimeo',
    15: 'Voipbuster',
    16: 'Youtube',
}

# for traffic identification
PREFIX_TO_TRAFFIC_ID = {
    # Chat
    'aim_chat_3a': 0,
    'aim_chat_3b': 0,
    'aimchat1': 0,
    'aimchat2': 0,
    'facebook_chat_4a': 0,
    'facebook_chat_4b': 0,
    'facebookchat1': 0,
    'facebookchat2': 0,
    'facebookchat3': 0,
    'hangout_chat_4b': 0,
    'hangouts_chat_4a': 0,
    'icq_chat_3a': 0,
    'icq_chat_3b': 0,
    'icqchat1': 0,
    'icqchat2': 0,
    'skype_chat1a': 0,
    'skype_chat1b': 0,
    # Email
    'email1a': 1,
    'email1b': 1,
    'email2a': 1,
    'email2b': 1,
    # File Transfer
    'ftps_down_1a': 2,
    'ftps_down_1b': 2,
    'ftps_up_2a': 2,
    'ftps_up_2b': 2,
    'sftp1': 2,
    'sftp_down_3a': 2,
    'sftp_down_3b': 2,
    'sftp_up_2a': 2,
    'sftp_up_2b': 2,
    'sftpdown1': 2,
    'sftpdown2': 2,
    'sftpup1': 2,
    'skype_file1': 2,
    'skype_file2': 2,
    'skype_file3': 2,
    'skype_file4': 2,
    'skype_file5': 2,
    'skype_file6': 2,
    'skype_file7': 2,
    'skype_file8': 2,
    # Streaming
    'vimeo1': 3,
    'vimeo2': 3,
    'vimeo3': 3,
    'vimeo4': 3,
    'youtube1': 3,
    'youtube2': 3,
    'youtube3': 3,
    'youtube4': 3,
    'youtube5': 3,
    'youtube6': 3,
    'youtubehtml5_1': 3,
    # Torrent
    'torrent01': 4,
    # VoIP
    'facebook_audio1a': 5,
    'facebook_audio1b': 5,
    'facebook_audio2a': 5,
    'facebook_audio2b': 5,
    'facebook_audio3': 5,
    'facebook_audio4': 5,
    'hangouts_audio1a': 5,
    'hangouts_audio1b': 5,
    'hangouts_audio2a': 5,
    'hangouts_audio2b': 5,
    'hangouts_audio3': 5,
    'hangouts_audio4': 5,
    'skype_audio1a': 5,
    'skype_audio1b': 5,
    'skype_audio2a': 5,
    'skype_audio2b': 5,
    'skype_audio3': 5,
    'skype_audio4': 5,
    # VPN: Chat
    'vpn_aim_chat1a': 6,
    'vpn_aim_chat1b': 6,
    'vpn_facebook_chat1a': 6,
    'vpn_facebook_chat1b': 6,
    'vpn_hangouts_chat1a': 6,
    'vpn_hangouts_chat1b': 6,
    'vpn_icq_chat1a': 6,
    'vpn_icq_chat1b': 6,
    'vpn_skype_chat1a': 6,
    'vpn_skype_chat1b': 6,
    # VPN: File Transfer
    'vpn_ftps_a': 7,
    'vpn_ftps_b': 7,
    'vpn_sftp_a': 7,
    'vpn_sftp_b': 7,
    'vpn_skype_files1a': 7,
    'vpn_skype_files1b': 7,
    # VPN: Email
    'vpn_email2a': 8,
    'vpn_email2b': 8,
    # VPN: Streaming
    'vpn_vimeo_a': 9,
    'vpn_vimeo_b': 9,
    'vpn_youtube_a': 9,
    # VPN: Torrent
    'vpn_bittorrent': 10,
    # VPN VoIP
    'vpn_facebook_audio2': 11,
    'vpn_hangouts_audio1': 11,
    'vpn_hangouts_audio2': 11,
    'vpn_skype_audio1': 11,
    'vpn_skype_audio2': 11,
}

ID_TO_TRAFFIC = {
    0: 'Chat',
    1: 'Email',
    2: 'File Transfer',
    3: 'Streaming',
    4: 'Torrent',
    5: 'Voip',
    6: 'VPN: Chat',
    7: 'VPN: File Transfer',
    8: 'VPN: Email',
    9: 'VPN: Streaming',
    10: 'VPN: Torrent',
    11: 'VPN: Voip',
}

In [11]:
from pathlib import Path

import click
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from scapy.compat import raw
from scapy.layers.inet import IP, UDP
from scapy.layers.l2 import Ether
from scapy.packet import Padding
from scipy import sparse



def remove_ether_header(packet):
    if Ether in packet:
        return packet[Ether].payload

    return packet


def mask_ip(packet):
    if IP in packet:
        packet[IP].src = '0.0.0.0'
        packet[IP].dst = '0.0.0.0'

    return packet


def pad_udp(packet):
    if UDP in packet:
        # get layers after udp
        layer_after = packet[UDP].payload.copy()

        # build a padding layer
        pad = Padding()
        pad.load = '\x00' * 12

        layer_before = packet.copy()
        layer_before[UDP].remove_payload()
        packet = layer_before / pad / layer_after

        return packet

    return packet


def packet_to_sparse_array(packet, max_length=1500):
    arr = np.frombuffer(raw(packet), dtype=np.uint8)[0: max_length] / 255
    if len(arr) < max_length:
        pad_width = max_length - len(arr)
        arr = np.pad(arr, pad_width=(0, pad_width), constant_values=0)

    arr = sparse.csr_matrix(arr)
    return arr


def transform_packet(packet):
    if should_omit_packet(packet):
        return None

    packet = remove_ether_header(packet)
    packet = pad_udp(packet)
    packet = mask_ip(packet)

    arr = packet_to_sparse_array(packet)

    return arr


def transform_pcap(path, output_path: Path = None, output_batch_size=10000):
    if Path(str(output_path.absolute()) + '_SUCCESS').exists():
        print(output_path, 'Done')
        return

    print('Processing', path)

    rows = []
    batch_index = 0
    for i, packet in enumerate(read_pcap(path)):
        arr = transform_packet(packet)
        if arr is not None:
            # get labels for app identification
            prefix = path.name.split('.')[0].lower()
            app_label = PREFIX_TO_APP_ID.get(prefix)
            traffic_label = PREFIX_TO_TRAFFIC_ID.get(prefix)
            row = {
                'app_label': app_label,
                'traffic_label': traffic_label,
                'feature': arr.todense().tolist()[0]
            }
            rows.append(row)

        # write every batch_size packets, by default 10000
        if rows and i > 0 and i % output_batch_size == 0:
            part_output_path = Path(str(output_path.absolute()) + f'_part_{batch_index:04d}.parquet')
            df = pd.DataFrame(rows)
            df.to_parquet(part_output_path)
            batch_index += 1
            rows.clear()

    # final write
    if rows:
        df = pd.DataFrame(rows)
        part_output_path = Path(str(output_path.absolute()) + f'_part_{batch_index:04d}.parquet')
        df.to_parquet(part_output_path)

    # write success file
    with Path(str(output_path.absolute()) + '_SUCCESS').open('w') as f:
        f.write('')

    print(output_path, 'Done')


@click.command()
@click.option('-s', '--source', help='path to the directory containing raw pcap files', required=True)
@click.option('-t', '--target', help='path to the directory for persisting preprocessed files', required=True)
@click.option('-n', '--njob', default=-1, help='num of executors', type=int)
def main(source, target, njob):
    data_dir_path = Path(source)
    target_dir_path = Path(target)
    target_dir_path.mkdir(parents=True, exist_ok=True)
    Parallel(n_jobs=njob)(
        delayed(transform_pcap)(pcap_path, target_dir_path / (pcap_path.name + '.transformed')) for pcap_path in
        sorted(data_dir_path.iterdir()))


if __name__ == '__main__':
    main()

Usage: ipykernel_launcher.py [OPTIONS]
Try 'ipykernel_launcher.py --help' for help.

Error: no such option: -f
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\software\anaconda\lib\site-packages\click\parser.py", line 416, in _process_opts
    self._match_long_opt(norm_long_opt, explicit_value, state)
  File "D:\software\anaconda\lib\site-packages\click\parser.py", line 326, in _match_long_opt
    raise NoSuchOption(opt, possibilities=possibilities, ctx=self.ctx)
click.exceptions.NoSuchOption: no such option: -f

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\software\anaconda\lib\site-packages\click\core.py", line 781, in main
    with self.make_context(prog_name, args, **extra) as ctx:
  File "D:\software\anaconda\lib\site-packages\click\core.py", line 700, in make_context
    self.parse_args(ctx, args)
  File "D:\software\anaconda\lib\site-packages\click\core.py", line 1045, in parse_args
    opts, args, param_order = parser.parse_args(args=args)
  File "D:\software\anaconda\lib\site-packages\click\parser.py", line 269, in par

TypeError: object of type 'NoneType' has no len()